In [1]:
# !pip install transformers==4.27.2 datasets==2.11.0 evaluate==0.4.0 rouge_score==0.1.2 loralib==0.1.1 peft==0.3.0 --quiet

In [2]:
# ! /usr/bin/python3 -m pip install -U tensorflow-cpu==2.10.0 --quiet

In [3]:
!pip install pyarrow<15
!pip install transformers tensorflow datasets evaluate rouge_score loralib peft -U --quiet

/bin/bash: line 1: 15: No such file or directory


In [4]:
# !pip install torch torchdata --quiet

In [5]:
from datasets import load_dataset, Dataset, table, DatasetDict
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import time
import evaluate
import pandas as pd
import numpy as np
# import torch

In [6]:
# data_path = "ccdv/arxiv-summarization"#: ("article", "abstract")
# data_path = "knkarthick/dialogsum"
data_path = "albertvillanova/meqsum"
dataset = load_dataset(data_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
# from transformers import DistilBertTokenizer, TFDistilBertModel
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
original_model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = original_model(encoded_input, decoder_input_ids=encoded_input.input_ids)


/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [8]:
# output[0]

In [9]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for param in model.trainable_weights:
        all_model_params += param.shape.num_elements()
        if param.trainable:
            trainable_model_params += param.shape.num_elements()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

# original_model.trainable_weights[0].trainable

trainable model parameters: 76961152
all model parameters: 76961152
percentage of trainable model parameters: 100.00%


In [10]:
from sklearn.model_selection import train_test_split
pd.DataFrame(dataset["train"])
dialogue = dataset["train"]["CHQ"]
summary = dataset["train"]["Summary"]

Evalutate before fine tuning

In [11]:
rouge = evaluate.load('rouge')

In [12]:
# def preprocess():
prompt = [f"""
Summarize the following conversation.

{dialogues}

Summary:
""" for dialogues in dialogue]
# prompt
# type(dialogue)
input = [tokenizer(prompts, return_tensors="tf") for prompts in prompt]

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


In [13]:
prompt[20]

'\nSummarize the following conversation.\n\nPrenatal exams for leigh syndrome. I am looking for information about having prenatal exams anmnio or chorionic villus sampling for NARP and leigh syndrome. Thank you in advance\n\nSummary:\n'

In [14]:
# input[20]

In [15]:
index = 20

output = tokenizer.decode(
    original_model.generate(
        input[index]["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt[index]}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary[index]}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

Prenatal exams for leigh syndrome. I am looking for information about having prenatal exams anmnio or chorionic villus sampling for NARP and leigh syndrome. Thank you in advance

Summary:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
Where can I find information on prenatal genetic tests for NARP and leigh syndrome?

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
W: I have a prenatal exam for leigh syndrome. Do you have any questions?


In [16]:
# !pip install scikit-learn
from sklearn.model_selection import train_test_split

In [17]:
train, test = train_test_split(dataset['train'].to_pandas(), test_size=0.1)

In [18]:
from tqdm import tqdm

In [19]:
dialogues = test[0:10]['CHQ']
human_baseline_summaries = test[0:10]['Summary']

original_model_summaries = []
# instruct_model_summaries = []

for _, dialogue in enumerate(tqdm(dialogues)):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="tf").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids,
                                                     generation_config=GenerationConfig(max_new_tokens=200)
                                                    )
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    # instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    # instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    # instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries'])
df

100%|██████████| 10/10 [00:48<00:00,  4.86s/it]


,human_baseline_summaries,original_model_summaries
0,What are the treatments for drop foot?,Do you have any other suggestions?
1,"What are the treatments for strabismus, includ...",I am looking for information about the develop...
2,What are the treatments for coughing up blood?,i want to know if i can get a throat infection
3,What health conditions are associated with cha...,"W: I'm sorry, but I'm not sure what to do with..."
4,Does Vamoose head lice treatment conform to FD...,Head lice treatment claims are in violation of...
5,Are there drug interactions between ginkgo and...,Ginkgo and Pycnogenol are not the same.
6,What are the treatments for Ankylosing Spondyl...,I am suffering from ankylosing spondylitis.
7,What causes foamy saliva?,A friend has developed a foam in her mouth tha...
8,Where can I find information on clinical trial...,I am a diabetic and have been diagnosed with t...
9,What are the treatments for hand tremor?,WHERE IS THE TURNING PART OF HEALTH?


In [20]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)].to_list(),
    use_aggregator=True,
    use_stemmer=True,
)

In [21]:
original_model_results

{'rouge1': 0.20541137878733787,
 'rouge2': 0.07429195804195804,
 'rougeL': 0.1748875600793759,
 'rougeLsum': 0.17412962236491647}

In [ ]:
# {'rouge1': 0.21006846995973436,
#  'rouge2': 0.09184877622377621,
#  'rougeL': 0.185532738090876,
#  'rougeLsum': 0.18673884743102717}

PEFT

In [22]:
ft_model_name = "google/flan-t5-small"
ft_tokenizer = AutoTokenizer.from_pretrained(ft_model_name)
ft_model = TFAutoModelForSeq2SeqLM.from_pretrained(ft_model_name)

/usr/local/lib/python3.10/dist-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [23]:
# train_testvalid = dataset['train'].select(range(10)).train_test_split(test_size=0.2, shuffle=True)
train_testvalid = dataset['train'].train_test_split(test_size=0.2, shuffle=True)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)

ttv_ds = DatasetDict({
'train': train_testvalid['train'],
'validation': test_valid['train'],
'test': test_valid['test']})#.set_format('numpy')


In [24]:
# print(ttv_ds.set_format('numpy'))

In [25]:
# np_ttv_ds = convert_split_to_numpy(tokeinzed_ttv_ds)
# np_ttv_ds

In [26]:
def preprocess_function(train_data):
    inputs = ["Summarize this text:" + doc for doc in train_data['CHQ']]
    model_inputs = ft_tokenizer(inputs, padding='max_length', truncation=True, max_length=200, return_tensors="np")
    labels = ft_tokenizer(text_target=train_data["Summary"], padding='max_length', truncation=True, max_length=200, return_tensors="np")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [27]:
tokenized_ttv_ds= ttv_ds.map(preprocess_function, batched=True, remove_columns= ['CHQ', 'Summary', 'File'])

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
# def convert_split_to_numpy(split):
#     input_ids = np.array(split['input_ids'])
#     attention_mask = np.array(split['attention_mask'])
#     labels = np.array(split['labels'])
#     # ds = Dataset.from_dict({
#     #     'input_ids': input_ids,
#     #     'attention_mask': attention_mask,
#     #     'labels': labels
#     # })
#     ds = DatasetDict({
#         'input_ids': input_ids,
#         'attention_mask': attention_mask,
#         'labels': labels
#     })#.set_format('numpy')
#     return ds

# # Assuming your DatasetDict is named dataset
# # Convert each split individually
# np_train = convert_split_to_numpy(tokenized_ttv_ds['train'])
# np_validation = convert_split_to_numpy(tokenized_ttv_ds['validation'])
# np_test = convert_split_to_numpy(tokenized_ttv_ds['test'])


In [29]:
tokenized_ttv_ds.set_format('numpy')

In [30]:
tokenized_ttv_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [31]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=ft_tokenizer, model=ft_model_name, return_tensors="tf")

In [32]:
# tf_train_set = ft_model.prepare_tf_dataset(
#     tokenized_ttv_ds['train'],
#     shuffle=True,
#     batch_size=10,
#     collate_fn=data_collator,
# )
# tf_valid_set = ft_model.prepare_tf_dataset(
#     tokenized_ttv_ds['validation'],
#     shuffle=True,
#     batch_size=10,
#     collate_fn=data_collator,
# )
# type(np_validation['input_ids'])

In [33]:
tf_train_set = tokenized_ttv_ds['train'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'label'],
    shuffle=True,
    batch_size=10,
    collate_fn=data_collator,
)
tf_valid_set = tokenized_ttv_ds['validation'].to_tf_dataset(
    columns=['input_ids', 'attention_mask', 'label'],
    shuffle=True,
    batch_size=10,
    collate_fn=data_collator,
)

In [34]:
# tokenized_ttv_ds
# list(tf_train_set)

In [35]:
# from datasets import load_dataset
# import tensorflow as tf

# # Assuming `datasets` is your DatasetDict
# # For demonstration, let's say datasets = load_dataset('your_dataset_name')

# def convert_to_tf_dataset(dataset, columns=['input_ids', 'attention_mask', 'labels']):
#     # Convert to TensorFlow dataset
#     dataset.set_format(type='tensorflow', columns=columns)
#     features = {x: dataset[x].to_tensor(default_value=0, shape=[None]) for x in columns[:-1]}  # input features
#     labels = dataset[columns[-1]].to_tensor(default_value=0, shape=[None])  # labels
#     tf_dataset = tf.data.Dataset.from_tensor_slices((features, labels))
#     return tf_dataset

# # Convert each split to TensorFlow datasets
# tf_train_dataset = convert_to_tf_dataset(datasets["train"])
# tf_validation_dataset = convert_to_tf_dataset(datasets["validation"])
# tf_test_dataset = convert_to_tf_dataset(datasets["test"])


In [36]:
rouge = evaluate.load("rouge")

In [37]:
from transformers import create_optimizer, AdamWeightDecay

opt = AdamWeightDecay(learning_rate=2e-6, weight_decay_rate=0.01)

ft_model.compile(optimizer=opt)

In [38]:
import numpy as np
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = ft_tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, ft_tokenizer.pad_token_id)
    decoded_labels = ft_tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != ft_tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_valid_set)

In [39]:
# train_dataset = tf.data.Dataset.from_tensor_slices((np_train['input_ids'], np_train['labels']))
# validation_dataset = tf.data.Dataset.from_tensor_slices((np_validation['input_ids'], np_validation['labels']))

In [41]:
ft_model.fit(tf_train_set,
             validation_data=tf_valid_set,
             epochs=1,
            #  callbacks=metric_callback,
             )

# ft_model.fit(train_dataset,
#              validation_data=validation_dataset,
#              epochs=3,
#             #  callbacks=metric_callback,
#              )

80/80 [==============================] - 34s 422ms/step - loss: 34.8460 - val_loss: 34.4384


In [42]:
# list(tf_train_set)

In [43]:
dialogues = test[0:10]['CHQ']
human_baseline_summaries = test[0:10]['Summary']

ft_model_summaries = []
# instruct_model_summaries = []

for _, dialogue in enumerate(tqdm(dialogues)):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="tf").input_ids

    ft_model_outputs = ft_model.generate(input_ids=input_ids,
                                                     generation_config=GenerationConfig(max_new_tokens=200)
                                                    )
    ft_model_text_output = tokenizer.decode(ft_model_outputs[0], skip_special_tokens=True)
    ft_model_summaries.append(ft_model_text_output)

    # instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    # instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    # instruct_model_summaries.append(instruct_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, ft_model_summaries))

df_ft = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'ft_model_summaries'])
df_ft

100%|██████████| 10/10 [01:57<00:00, 11.72s/it]


,human_baseline_summaries,ft_model_summaries
0,What are the treatments for drop foot?,Drop foot should be done to help you get better.
1,"What are the treatments for strabismus, includ...",I am looking for information about the develop...
2,What are the treatments for coughing up blood?,i want to know if i can get a sneezing cough
3,What health conditions are associated with cha...,"W: I'm sorry, but I'm not sure what to do with..."
4,Does Vamoose head lice treatment conform to FD...,- Vamousse head lice treatment labe claims
5,Are there drug interactions between ginkgo and...,Ginkgo and Pycnogenol are not the same.
6,What are the treatments for Ankylosing Spondyl...,I am suffering from ankylosing spondylitis.
7,What causes foamy saliva?,A friend has developed a foam in her mouth tha...
8,Where can I find information on clinical trial...,I am a diabetic and I am a member of the JDRF ...
9,What are the treatments for hand tremor?,WHERE IS THE HEALTH HEALTH HEALTH HEALTH HEALT...


In [44]:
ft_model_results = rouge.compute(
    predictions=ft_model_summaries,
    references=human_baseline_summaries[0:len(ft_model_summaries)].to_list(),
    use_aggregator=True,
    use_stemmer=True,
)

In [45]:
ft_model_results

{'rouge1': 0.21006846995973436,
 'rouge2': 0.09184877622377621,
 'rougeL': 0.185532738090876,
 'rougeLsum': 0.18673884743102717}

In [ ]:
# ft_model.save('/content/model_parmas1')

In [ ]:
#  !zip -r /content/sample_data/model_params1.zip /content/model_parmas1/